In [1]:
!pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 13.2 MB/s eta 0:00:00


In [14]:
pip install --upgrade keras-tuner


In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np


In [3]:
print(tf.__version__)

2.13.0


In [4]:
fashion_mnist = keras.datasets.fashion_mnist

In [5]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [6]:
train_images=train_images/255.0
test_images=test_images/255.0

In [7]:
train_images[0].shape

(28, 28)

In [8]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [9]:
def build_model(hp):
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])

  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

  return model

In [17]:
from keras_tuner import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameter


In [18]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [19]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 00m 25s]
val_accuracy: 0.9083333611488342

Best val_accuracy So Far: 0.9083333611488342
Total elapsed time: 00h 02m 30s


In [20]:
model=tuner_search.get_best_models(num_models=1)[0]

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 24, 24, 32)        832       
                                                                 
 conv2d_1 (Conv2D)           (None, 22, 22, 32)        9248      
                                                                 
 flatten (Flatten)           (None, 15488)             0         
                                                                 
 dense (Dense)               (None, 112)               1734768   
                                                                 
 dense_1 (Dense)             (None, 10)                1130      
                                                                 
Total params: 1745978 (6.66 MB)
Trainable params: 1745978 (6.66 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [22]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 8s 4ms/step - loss: 0.1625 - accuracy: 0.9404 - val_loss: 0.2582 - val_accuracy: 0.9102
Epoch 5/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.1245 - accuracy: 0.9534 - val_loss: 0.2674 - val_accuracy: 0.9173
Epoch 6/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0933 - accuracy: 0.9658 - val_loss: 0.2886 - val_accuracy: 0.9168
Epoch 7/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0709 - accuracy: 0.9733 - val_loss: 0.3263 - val_accuracy: 0.9138
Epoch 8/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0541 - accuracy: 0.9799 - val_loss: 0.3976 - val_accuracy: 0.9112
Epoch 9/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0403 - accuracy: 0.9856 - val_loss: 0.4424 - val_accuracy: 0.9093
Epoch 10/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.0382 - accuracy: 0.9863 - val_loss: 0.4768 - val_accuracy